In [2]:
import os
import sys
import pandas as pd
import numpy as np
import boto3
import feather
import tempfile
from time import time

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir,'src'))

from d01_data.ingestion_xtdb import ingest_xtdb
from d02_intermediate.clean_xtdb import *
from d00_utils.db_utils import *
from d00_utils.s3_utils import *

db_table = 'metadata'
credentials_file = os.path.join(os.path.expanduser('~'), '.psql_credentials.json')

In [3]:
io_clean = dbReadWriteClean()
io_clean.list_tables()

['meta_lite', 'measurement_abstract_rpt', 'a_measgraphref', 'a_measgraphic', 'dm_spain_view_study_summary', 'a_modvolume', 'instance_filename']


In [4]:
t1 = time()
meta_lite = io_clean.get_table('meta_lite')
print(time() - t1)

/home/wiebke_toussaint/anaconda3/envs/dssg/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


29.501980781555176


In [47]:
s3keys = get_matching_s3_keys('cibercv','100000/a_6MI50JZH')

In [48]:
for k in s3keys: print(k.split('/')[1]+'_raw')

a_6MI50JZH.dcm_raw


In [42]:
'100000/a_6MI50JZH'.split('/')

['100000', 'a_6MI50JZH']

In [51]:
def extract_imgs_from_dicom(bucket, prefix='', suffix='.dcm'):
    """
    Get all the keys with a specific suffix from a s3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with this prefix (optional).
    :param suffix: Only fetch objects whose keys end with this suffix, default='dcm'
    :return: list of keys
    """
    s3 = boto3.client('s3')

    keys = []
    kwargs = {'Bucket': bucket}

    data_dir = os.path.join(projectdir, 'data', '01_raw','dicom_images')
    os.makedirs(data_dir, exist_ok=True)

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            if obj['Key'].startswith(prefix) and obj['Key'].endswith(suffix):
                keys.append(obj['Key'])

                outrawfilename = prefix.split('/')[1]+'_raw'

                # Download and convert dicom file to raw format
                tmp = tempfile.NamedTemporaryFile()
                s3.download_file(bucket, obj['Key'], tmp.name)
                os.system('gdcmconv -w ' + tmp.name + " " + os.path.join(data_dir, outrawfilename))

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return keys

In [52]:
extract_imgs_from_dicom('cibercv','100000/a_6MI50JZH')

TypeError: makedirs() missing 1 required positional argument: 'name'